# 10_序列的修改、散列和切片

- `gensim`: 使用 numpy 和 scipy 实现了用于处理自然语言和检索信息的向量空间模型。
- `reprlib`：生成有限长度的表现形式。

In [17]:
###### from math import sqrt
from array import array
import reprlib
import numbers
import functools

class Vector:
    """ 多维向量 """
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    def __repr__(self):
        """ 使用 reprlib 显示有限长度的输出 """
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return "Vector({})".format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return "({}, {})".format(*components)
        
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
        
    def __iter__(self):
        return iter(self._components)
    
    def __hash__(self):
        # hashes = (hash(x) for x in self._components)
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        """ Return the Euclidean distance, sqrt(x*x + y*y). """
        return sqrt(sum(x*x for x in self))
        
    def __bool__(self):
        return bool(abs(self))
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            # 方案1
            if len(self) != len(other):
                return False
            for a,b in zip(self, other):
                if a != b:
                    return False
            # 方案2, 采用 all + zip
            # return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        else:
            raise ValueError("obj is not Vector obj!")
    
#     def __add__(self, other):
#         x = self.x + other.x
#         y = self.y + other.y
#         return Vector(x, y)
    
#     def __mul__(self, scalar):
#         return Vector(self.x * scalar, self.y * scalar)
    
#     def angle(self):
#         return atan2(self.y, self.x)
    
    @classmethod
    def frombytes(cls, octests):
        typecode = chr(octests[0])
        memv = memoryview(octests[1:]).cast(typecode)
        return cls(memv)
    
    # 可切片序列
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        """ 可接受 index """
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
            
    # 动态存储属性
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos =cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} obj has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
#         if len(name) == 1:
#             # 判断属性名是否包含在shortcut_names
#             if name in cls.shortcut_names:
#                 error = ""
#                 # error = "readonly attribute {attr_name!r}"
#             # 属性名小写
#             elif name.islower():
#                 error = "can't set attributes 'a' to 'z' in {cls_name!r}"
#             else:
#                 error = ''
#             if error:
#                 msg = error.format(cls_name=cls.__name__, attr_name=name)
#                 raise AttributeError(msg)
        super().__setattr__(name, value)

In [2]:
# 测试
import reprlib
lst = [x for x in range(100)]
lst = reprlib.repr(lst)
print(lst)
print(lst.find('['))
lst1 = lst[lst.find('['):-1]
lst1

s = 'asd'
s.islower()

[0, 1, 2, 3, 4, 5, ...]
0


True

In [3]:
# 切片示意
v1 = Vector(range(10))
print(f"len(v1) \t {len(v1)}")
print(f"v1[0] \t\t {v1[0]}")
print(f"v1[:2] \t\t {v1[:2]}")
print(f"v1[1:3] \t {v1[1:3]}")
try:
    print(f"v1[1,2] \t {v1[1,2]}")
except Exception as e:
    print(f"Error: {e}")

v1[1:7]

len(v1) 	 10
v1[0] 		 0.0
v1[:2] 		 (0.0, 1.0)
v1[1:3] 	 (1.0, 2.0)
Error: Vector indices must be integers


Vector([1.0, 2.0, 3.0, 4.0, 5.0, ...])

In [4]:
# 动态存储属性
v = Vector(range(5))
print(f"v \t {v}")
print(f"v.x \t {v.x}")

v.x = 10
print(f"v.x \t {v.x}")
print(f"v \t {v}")

v 	 (0.0, 1.0)
v.x 	 0.0
v.x 	 10
v 	 (0.0, 1.0)


## 10.1 协议和鸭子类型

- 协议：不强制要求，可以实现部分方法即可。
- 鸭子类型：
- Python 协议：
    * 序列协议： `__len__` | `__getitem__` | `__setitem__`(支持 `v[0]=1` 赋值) ；
    * 动态属性：`__getattr__` | `__setattr__` | `__delattr__`；

In [5]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) 
                       for suit in self.suits 
                       for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [6]:
card = FrenchDeck()
print(f"len(card) \t {len(card)}")
print(f"card[10] \t {card[10]}")
print(f"card[10:12] \t {card[10:12]}")

len(card) 	 52
card[10] 	 Card(rank='Q', suit='spades')
card[10:12] 	 [Card(rank='Q', suit='spades'), Card(rank='K', suit='spades')]


## 10.2 切片原理

In [7]:
class MySeq:
    def __getitem__(self, index):
        return index

In [8]:
s = MySeq()
print(f"s[1] \t {s[1]}")
print(f"s[1:3] \t {s[1:3]}")
print(f"s[1:3:2] \t {s[1:3:2]}")
print(f"s[1:3:2, 9] \t {s[1:3:2, 9]}")
print(f"s[1:3:2, 7:9] \t {s[1:3:2, 7:9]}")

s[1] 	 1
s[1:3] 	 slice(1, 3, None)
s[1:3:2] 	 slice(1, 3, 2)
s[1:3:2, 9] 	 (slice(1, 3, 2), 9)
s[1:3:2, 7:9] 	 (slice(1, 3, 2), slice(7, 9, None))


In [9]:
# 列表测试
s = [i for i in range(10)]
print(f"s[1] \t\t {s[1]}")
print(f"s[1:3] \t\t {s[1:3]}")
print(f"s[1:3:2] \t {s[1:3:2]}")
try:
    print(f"s[1:3:2, 9] \t {s[1:3:2, 9]}")
    print(f"s[1:3:2, 7:9] \t {s[1:3:2, 7:9]}")
except Exception as e:
    print(f"Error: {e}")

s[1] 		 1
s[1:3] 		 [1, 2]
s[1:3:2] 	 [1]
Error: list indices must be integers or slices, not tuple


In [10]:
print(f"slice \t {slice}")
print(f"dir(slice) \n {dir(slice)}")
print(f"slice.indices \t {slice.indices}")

print(f"slice(None, 10, 2).indices(5) \t {slice(None, 10, 2).indices(5)}")
print(f"slice(-3, None, None).indices(5) \t {slice(-3, None, None).indices(5)}")

slice 	 <class 'slice'>
dir(slice) 
 ['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']
slice.indices 	 <method 'indices' of 'slice' objects>
slice(None, 10, 2).indices(5) 	 (0, 5, 2)
slice(-3, None, None).indices(5) 	 (2, 5, 1)


## 10.3 散列和快速等值测试

### 归约函数
- reduce
    * `reduce(fn, lst, initializer)`,其中 fn 接受两个参数
        * 如果 lst 为空，可能会报 TypeError: reduce() of empty sequence with no initial value;
        * initializer，lst 为空时，返回的结果值；对 `+` | `|` | `^`，为0； `*` | `&`, 为1；
    * 执行原理：
        > -> `r1 = fn(lst[0],lst[1])`  
        > -> `r2 = fn(r1,lst[2])`  
        > -> `r3 = fn(r2,lst[3])`  
        > -> ...  
        > -> `rn = fn(r(n-1),lst[n])`   
        > -> 返回 `rn` 
- sum
- all
- any

In [12]:
# 利用 reduce 计算阶乘
import functools
res = functools.reduce(lambda a,b:a*b, range(1, 6))
res

120

### 位运算符

- `&`: 按位与运算符
    * 参与运算的两个值,如果两个相应位都为1,则该位的结果为1,否则为0	
    * (a & b) 输出结果 12 ，二进制解释： 0000 1100
- `|`: 按位或运算符
    * 只要对应的二个二进位有一个为1时，结果位就为1	
    * (a | b) 输出结果 61 ，二进制解释： 0011 1101
- `^`: 按位异或运算符
    * 当两对应的二进位相异时，结果为1	
    * (a ^ b) 输出结果 49 ，二进制解释： 0011 0001
- `~`: 按位取反运算符
    * 对数据的每个二进制位取反,即把1变为0,把0变为1	
    * (~a ) 输出结果 -61 ，二进制解释： 1100 0011， 在一个有符号二进制数的补码形式。
- `<<`: 左移动运算符
    * 运算数的各二进位全部左移若干位，由`<<`右边的数指定移动的位数，高位丢弃，低位补0	
    * a << 2 输出结果 240 ，二进制解释： 1111 0000
- `>>`: 右移动运算符
    * 把 `>>` 左边的运算数的各二进位全部右移若干位，`>>`右边的数指定移动的位数	
    * a >> 2 输出结果 15 ，二进制解释： 0000 1111

In [13]:
# 计算累计 异或^
n = 0
for i in range(1, 6):
    n ^= i
n

1

In [14]:
# 使用 lambda + reduce
import functools
res = functools.reduce(lambda a,b:a^b, range(6))
res

1

In [16]:
# 使用 operator + reduce
import functools
import operator
res = functools.reduce(operator.xor, range(6))
res

1

- `zip` 

In [7]:
print(f"zip(range(3), 'ABC') \t\t\t\t\t {zip(range(3), 'ABC')}")
print(f"list(zip(range(3), 'ABC')) \t\t\t\t {list(zip(range(3), 'ABC'))}")
print(f"list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3])) \t {list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))}")  # 按照最短的组合

from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))  # 按照最长的组合，不足的补0

zip(range(3), 'ABC') 					 <zip object at 0x000001F037213BC8>
list(zip(range(3), 'ABC')) 				 [(0, 'A'), (1, 'B'), (2, 'C')]
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3])) 	 [(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]


[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]